# READ BEFORE EXECUTING
https://www.themoviedb.org/
* *You will need to request an api key from the above website*
* *You will need to create a config.py file* to hold your api key for TMDB. 
    * Have it read tmdb_key = "YOUR KEY"
* *You will need to unzip "movie_title.csv.zip".* 
    * movie_title.csv was too large for me to push. 
    * You can store it in you local branch, then delete it once you are done with it. 
    * When you unzip it, the csv file will populate in the same folder (Resources) as the zip...which is where the script will think it is.
<br><br>
* "PUT YOUR YEAR HERE"
    * This is in the 3rd cell down. *You will need to update this with your assigned year.*
* You will need to do two pulls: 
    * 1) to get the TMDB ID
    * 2) to get the movie info based off the TMBD ID.
    * This will take a while
        * I'm guessing let it run and check back in an hour to make sure there are now errors
        * If there are no errors, it should be able to run clean through.

#### IF THERE ARE ERRORS
* Please take a screenshot of the ENTIRE error message
* For Pull 1:
    * print(response_tmdb_id.count())
    * print(error_count.count())
    * Add the 2 count numbers together
    * print(movies[THE NUMBER HERE])
        * That should be the title that thew it off
* For Pull 2:
    * Please go into the cell below where the CSV is saved and run it
    * Open it up and see what the last title that rna successfully and see what comes next
        * That should be the title that thew it off

#### Make sure to zip all 3 CSVs this script will create. 
* We need all 3 so we can tell what was dropped when/where.
* If they are not zipped, there might be issues pushing to the main branch due to size. You can right click and there should be a compress or zip option.

### FOR THE LOVE OF ALL THAT IS GOOD AND HOLY MAKE SURE YOU ARE ON YOUR BRANCH OF GIT (please and thank you!)

In [1]:
import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings('ignore')
import requests
from config import tmdb_key

In [2]:
file_path = 'Resources/movie_title.csv'

In [3]:
# *****PUT YOUR YEAR HERE*****

year = 2016 # <------ YOUR YEAR GOES HERE!!!!!

In [4]:
movie_titles_df = pd.read_csv(file_path)
movie_titles_df.head()

movies_df = movie_titles_df[['tconst', 'primaryTitle', 'startYear']]
movies_df = movies_df[(movies_df.startYear == year)]

beginning_number = movies_df.tconst.count()
movies_df.head()

,tconst,primaryTitle,startYear
18,tt0308078,Les bâtisseurs,2016
19,tt0315642,Wazir,2016
27,tt0364201,Aman Ke Farishtey,2016
31,tt0376479,American Pastoral,2016
38,tt0443533,The History of Love,2016


# Making the dataframe into a list + editing for error prevention

In [5]:
# change title name to have + instead of ' '
movies_df['primaryTitle'] = movies_df['primaryTitle'].str.replace(" ", "+")


# ******Error 1: need to remove # from the beginning of titles for TMDB to work


# variable cause starswith() wasn't happy with '#'
pound_sign = '#'

# make dataframe for pound sign = True (startswith() returns True/False)
replace_pound_df = movies_df.iloc[:, 0:3]
replace_pound_df.primaryTitle = replace_pound_df.primaryTitle.str.startswith(pound_sign)

# make df for ONLY the True values + primaryTitle from movie_df
pound_true_df = replace_pound_df.loc[replace_pound_df.primaryTitle == True]
pound_true_df['TITLE'] = movies_df['primaryTitle']

# Fix titles to not have # in the front & clean up columns
pound_true_df['TITLE'] = pound_true_df['TITLE'].str.replace(pound_sign, "")
pound_true_clean_df = pound_true_df.drop(columns=['primaryTitle', 'startYear'])
pound_true_clean_df = pound_true_clean_df.rename(columns={'TITLE': 'primaryTitle'})

# Merge 2 dfs, replace blank primaryTitle_y values with na so you can do fillna into a 
# nice new clean has correct info column & delete primaryTitle_y/x
titles_combined_df = pd.merge(movies_df, pound_true_clean_df, how='outer', on='tconst')
titles_combined_df['primaryTitle_y'] = titles_combined_df['primaryTitle_y'].str.replace(" ", "nan")
titles_combined_df["primaryTitle"] = titles_combined_df["primaryTitle_y"].fillna(titles_combined_df["primaryTitle_x"])
titles_fixed_df = titles_combined_df.drop(columns=['primaryTitle_y', 'primaryTitle_x'])


# FINALLY make movie titles into a list so you can run it
movies = titles_fixed_df['primaryTitle'].tolist()


# TMDB call 1 - get TMDB ID numbers

In [6]:
url_tmdb_id = "https://api.themoviedb.org/3/search/movie?api_key=" + tmdb_key + "&query="

response_tmdb_id = []
str_year = "&y=" + str(year)

error_count = 0

for movie in movies: 
    try: 
        movie_data = requests.get(url_tmdb_id + movie + str_year).json()
        if (movie_data['total_results'] == 1):
            response_tmdb_id.append(movie_data['results'][0]['id'])       
        else:
            error_count += 1
    except (IndexError, KeyError, ValueError):
        error_count += 1

print(f"A total of {error_count} movies could not be found.")

SSLError: HTTPSConnectionPool(host='api.themoviedb.org', port=443): Max retries exceeded with url: /3/search/movie?api_key=96eae9c9a1bd0a1459287ca0bca1e8da&query=The+Sun+at+Midnight&y=2016 (Caused by SSLError(SSLEOFError(8, 'EOF occurred in violation of protocol (_ssl.c:852)'),))

In [7]:
print(error_count)

3452


In [20]:
len(response_tmdb_id)


1902

#### Save results as a CSV

In [12]:
file_outpath = f"Resources/TMDB_pull_1_{year}_error_count{error_count}.csv"

TMDB_df = pd.DataFrame(response_tmdb_id,columns=['ID'],dtype=object)
TMDB_df.to_csv(file_outpath)

# TMDB call 2 - use TMDB ID numbers to get movie info

In [13]:
url_tmdb_movie = "https://api.themoviedb.org/3/movie/"

# Make columns to import info into
TMDB_df['imdb_id'] = " "
TMDB_df['release_date'] = " "
TMDB_df['budget'] = " "
TMDB_df['revenue'] = " "
TMDB_df['genres'] = " "
TMDB_df['original_language'] = " "
TMDB_df['original_title'] = " "
TMDB_df['origin_country'] = " "
TMDB_df['production_countries name'] = " "
TMDB_df['spoken_languages name'] = " "

In [14]:
error_count_info = 0

for index, row in TMDB_df.iterrows(): 
    movie_data = requests.get(url_tmdb_movie + str(TMDB_df.ID[index]) + "?api_key=" + tmdb_key).json()
    try:
        TMDB_df.loc[index, 'imdb_id'] = movie_data['imdb_id']
        TMDB_df.loc[index, 'release_date'] = movie_data['release_date']
        TMDB_df.loc[index, 'budget'] = movie_data['budget']
        TMDB_df.loc[index, 'revenue'] = movie_data['revenue']
        TMDB_df.loc[index, 'original_language'] = movie_data['spoken_languages'][0]['name']
        TMDB_df.loc[index, 'original_title'] = movie_data['original_title']
        TMDB_df.loc[index, 'origin_country'] = movie_data['production_countries'][0]['iso_3166_1']
        TMDB_df.loc[index, 'production_countries name'] = movie_data['production_countries'][0]['name']
        TMDB_df.loc[index, 'spoken_languages name'] = movie_data['spoken_languages'][0]['name']
        TMDB_df.loc[index, 'genres'] = movie_data['genres'][0]['name']    
    except (IndexError, KeyError):
        error_count_info +=1

#### Save results as a CSV

In [15]:
file_outpath_2 = f"Resources/TMDB_pull_2_{year}_error_count{error_count_info}.csv"

TMDB_df.to_csv(file_outpath_2)

# CLEANING if budget = 0, revenue = 0, IMDB_id not found
* This is to help keep the file size down by dropping rows we cannot use or cannot match up

In [16]:
movie_info_pulled_df = TMDB_df.copy()
movie_info_pulled_df.head()

,ID,imdb_id,release_date,budget,revenue,genres,original_language,original_title,origin_country,production_countries name,spoken_languages name
0,532608,tt0470936,2016-05-04,0,0,,,"Tak erkir, tsurt dzmer",,,
1,511526,tt0821486,2016-05-01,0,0,,,,,,
2,339116,tt0825283,2016-09-30,0,4218,Action,English,Maximum Ride,US,United States of America,English
3,347689,tt0837156,2016-03-15,0,0,Adventure,English,Pee-wee's Big Holiday,US,United States of America,English
4,511054,tt0918520,2016-12-01,0,0,,,,,,


In [17]:
movie_info_pulled_df = movie_info_pulled_df[movie_info_pulled_df.budget != 0]
movie_info_pulled_df = movie_info_pulled_df[movie_info_pulled_df.revenue != 0]
movie_info_pulled_df = movie_info_pulled_df.dropna(subset=['imdb_id'])

final_number = movie_info_pulled_df.imdb_id.count()

#### Save results as a CSV

In [18]:
total_errors = beginning_number - final_number

file_outpath_FINAL = f"Resources/TMDB_pull_FINAL_{year}_dropped_movies_{total_errors}.csv"
movie_info_pulled_df.to_csv(file_outpath_FINAL)

In [19]:
movie_info_pulled_df


,ID,imdb_id,release_date,budget,revenue,genres,original_language,original_title,origin_country,production_countries name,spoken_languages name
9,318850,tt1002563,2016-03-10,18500000,6490401,Drama,English,The Young Messiah,US,United States of America,English
61,9493,tt0096320,1988-12-09,15000000,216614388,Comedy,English,Twins,US,United States of America,English
69,341006,tt1082807,2016-03-17,5000000,10166820,Action,English,The Belko Experiment,CO,Colombia,English
71,202769,tt1911661,2010-09-16,150000,10000,Documentary,Italiano,Vedozero,IT,Italy,Italiano
103,316152,tt1124037,2016-06-24,50000000,25035950,Action,English,Free State of Jones,US,United States of America,English
...,...,...,...,...,...,...,...,...,...,...,...
1804,363579,tt4914580,2016-09-07,9620000,53165427,Action,English,밀정,KR,South Korea,English
1842,407887,tt4939066,2016-07-27,12700000,49196532,History,English,인천상륙작전,KR,South Korea,English
1847,394684,tt4944596,2016-10-28,4000000,1598584,Thriller,Español,Que Dios nos perdone,ES,Spain,Español
1859,409502,tt4950110,2016-10-21,1500000,2000000,Drama,English,I'm Not Ashamed,US,United States of America,English
